# Vertex AI Custom Training with Docker

This notebook demonstrates how to build, push, and run a custom training job on Vertex AI using Docker with the correct region configuration.

## 1. Initialize Vertex AI

First, let's initialize Vertex AI with the correct project and region settings.

In [ ]:
import sys
sys.path.append('..')
import vertexai_start
import os

PROJECT_ID = os.getenv("VERTEX_PROJECT_ID")
REGION = os.getenv("VERTEX_REGION")

print(f"Project ID: {PROJECT_ID}")
print(f"Region: {REGION}")

## 2. Build and Push Docker Image

**Important**: The image URI uses `us-central1` region, not `europe-west4`. This fixes the original error.

In [ ]:
# Correct image URI with us-central1 region
IMAGE_URI = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/custom-training/iris-custom-training:latest"
print(f"Image URI: {IMAGE_URI}")

# Note: This assumes the Artifact Registry repository 'custom-training' exists
# You may need to create it first:
# gcloud artifacts repositories create custom-training --repository-format=docker --location=us-central1

In [ ]:
# Build and push the Docker image
# Run this in the terminal from the iris_custom_training_artifact_registry directory:
# ./build_and_push.sh

print("To build and push the image, run the following command in the terminal:")
print("cd iris_custom_training_artifact_registry")
print("./build_and_push.sh")

## 3. Submit Custom Training Job

Now we can submit the custom training job using the correct image URI.

In [ ]:
from google.cloud import aiplatform

# Create custom job
job = aiplatform.CustomJob(
    display_name="iris-custom-training-job",
    worker_pool_specs=[
        {
            "machine_spec": {
                "machine_type": "n1-standard-4",
            },
            "replica_count": 1,
            "container_spec": {
                "image_uri": IMAGE_URI,
            },
        }
    ],
)

print(f"Created job with image: {IMAGE_URI}")

In [ ]:
# Submit the job (uncomment to run)
# job.run(
#     service_account=None,
#     timeout=3600,
#     restart_job_on_user_error=False,
# )

print("Job configuration ready. Uncomment the above code to submit the job.")

## Key Fixes Applied

1. **Region Consistency**: Changed from `europe-west4` to `us-central1` to match the Vertex AI configuration.
2. **Correct Image URI**: Using `us-central1-docker.pkg.dev` instead of `europe-west4-docker.pkg.dev`.
3. **Proper Scripts**: Created build and submission scripts to automate the workflow.

## Prerequisites

Before running this workflow, ensure:

1. Artifact Registry repository exists:
   ```bash
   gcloud artifacts repositories create custom-training --repository-format=docker --location=us-central1
   ```

2. Docker is configured for Artifact Registry:
   ```bash
   gcloud auth configure-docker us-central1-docker.pkg.dev
   ```

3. Appropriate IAM permissions for Vertex AI and Artifact Registry.